# MTAOS addAberration

Notebook to show how the addAberration command in MTAOS works.

Notes
=====

* Need w_2021_12 to run, w_13 (and probably higher versions) won't run, due to changes in lsst.afw.cameraGeom
* need MTAOS, wep, ofc
* best run on teststand, not lsp, because MTAOS needs to import dds library

In [ ]:
import numpy as np

from lsst.ts.MTAOS.CollOfListOfWfErr import CollOfListOfWfErr
from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory
from lsst.ts.wep.Utility import CamType
from lsst.ts.ofc.Utility import getConfigDir, DofGroup
from lsst.ts.ofc.ctrlIntf.FWHMSensorData import FWHMSensorData
from lsst.ts.wep.ctrlIntf.MapSensorNameAndId import MapSensorNameAndId
from lsst.ts.wep.ctrlIntf.SensorWavefrontError import SensorWavefrontError
from lsst.ts.wep.Utility import FilterType
from lsst.ts.wep.bsc.CamFactory import CamFactory

In [ ]:
comcam = CamFactory.createCam(CamType.ComCam)

In [ ]:
comcam.getWfsCcdList()

In [ ]:
ofc = OFCCalculationFactory.getCalculator(CamType.ComCam, None)

In [ ]:
detectorList = [
            "R22_S02",
            "R22_S12",
            "R22_S22",
            "R22_S01",
            "R22_S11",
            "R22_S21",
            "R22_S00",
            "R22_S10",
            "R22_S20",
        ]

In [ ]:
sensorIdList = ofc.ztaac.mapSensorNameToId(detectorList)

In [ ]:
intrinsic_zk = ofc.ztaac.dataShare.getIntrinsicZk(FilterType.REF, ofc.ztaac.dataShare.getFieldIdx(detectorList))

## We adopt Noll indexes

In [ ]:
# focus term
intrinsic_zk[:,0:1]

In [ ]:
intrinsic_zk[:,0:1] += 0.1 ## add 0.1 um of defocus

In [ ]:
# oblique astigmatism
intrinsic_zk[:,1:2]

In [ ]:
intrinsic_zk[:,1:2] += 0.1 # add 0.1 um of oblique astigmatism

In [ ]:
# Vertical astigmatism
intrinsic_zk[:,2:3]

In [ ]:
intrinsic_zk[:,2:3] += 0.1 # add 0.1 um of vertical astigmatism

In [ ]:
# Vertical coma
intrinsic_zk[:,3:4]

In [ ]:
intrinsic_zk[:,3:4] += 0.1  # add 0.1 um of vertical coma

In [ ]:
# Horizontal coma
intrinsic_zk[:,4:5]

In [ ]:
intrinsic_zk[:,4:5] += 0.1  # add 0.1 um of horizontal coma

In [ ]:
# Vertical trefoil
intrinsic_zk[:,5:6]

In [ ]:
intrinsic_zk[:,5:6] += 0.1  # add 0.1 um of vertical trefoil

In [ ]:
listOfWfErr = []
for sensorId, annularZernikePoly in zip(sensorIdList, intrinsic_zk):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr.append(sensorWavefrontError)


In [ ]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.calculateCorrections(listOfWfErr)

In [ ]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

In [ ]:
m2HexapodCorrection.getCorrection()[2] + cameraHexapodCorrection.getCorrection()[2]

In [ ]:
intrinsic_zk[:,0:1] -= 0.1

In [ ]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.resetOfcState()

In [ ]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

In [ ]:
listOfWfErr = []
for sensorId, annularZernikePoly in zip(sensorIdList, intrinsic_zk):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr.append(sensorWavefrontError)


In [ ]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.calculateCorrections(listOfWfErr)

In [ ]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

# Checking if operations commute

Basically I want to verify that if I add coma-x and coma-y is equivalent to add coma-y and coma-x

In [ ]:
ofc_1 = OFCCalculationFactory.getCalculator(CamType.ComCam, None)
ofc_2 = OFCCalculationFactory.getCalculator(CamType.ComCam, None)

In [ ]:
intrinsic_zk = ofc.ztaac.dataShare.getIntrinsicZk(FilterType.REF, ofc.ztaac.dataShare.getFieldIdx(detectorList))

In [ ]:
add_coma_x = np.array(intrinsic_zk, copy=True)
add_coma_y = np.array(intrinsic_zk, copy=True)

In [ ]:
add_coma_x[:,3:4] += 0.1  # add 0.1 um of vertical coma
add_coma_y[:,4:5] += 0.1  # add 0.1 um of horizontal coma

In [ ]:
listOfWfErr_comax = []
listOfWfErr_comay = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma_x):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_comax.append(sensorWavefrontError)


for sensorId, annularZernikePoly in zip(sensorIdList, add_coma_y):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_comay.append(sensorWavefrontError)


In [ ]:
(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc_1.calculateCorrections(listOfWfErr_comax)

(
    m2HexapodCorrection_cy1,
    cameraHexapodCorrection_cy1,
    m1m3Correction_cy1,
    m2Correction_cy1,
) = ofc_1.calculateCorrections(listOfWfErr_comay)

In [ ]:
(
    m2HexapodCorrection_cy2,
    cameraHexapodCorrection_cy2,
    m1m3Correction_cy2,
    m2Correction_cy2,
) = ofc_2.calculateCorrections(listOfWfErr_comay)

(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc_2.calculateCorrections(listOfWfErr_comax)

In [ ]:
print(m2HexapodCorrection_cx1.getCorrection())
print(cameraHexapodCorrection_cx1.getCorrection())
print(m1m3Correction_cx1.getZForces())
print(m2Correction_cx1.getZForces())

In [ ]:
print(m2HexapodCorrection_cy1.getCorrection())
print(cameraHexapodCorrection_cy1.getCorrection())
print(m1m3Correction_cy1.getZForces())
print(m2Correction_cy1.getZForces())

In [ ]:
print(m2HexapodCorrection_cx2.getCorrection())
print(cameraHexapodCorrection_cx2.getCorrection())
print(m1m3Correction_cx2.getZForces())
print(m2Correction_cx2.getZForces())

In [ ]:
print(m2HexapodCorrection_cy2.getCorrection())
print(cameraHexapodCorrection_cy2.getCorrection())
print(m1m3Correction_cy2.getZForces())
print(m2Correction_cy2.getZForces())

In [ ]:
np.array(m2HexapodCorrection_cx1.getCorrection())+np.array(m2HexapodCorrection_cy1.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_cx2.getCorrection())+np.array(m2HexapodCorrection_cy2.getCorrection())

# Adding and removing the same factor

In [ ]:
ofc_1 = OFCCalculationFactory.getCalculator(CamType.ComCam, None)

In [ ]:
intrinsic_zk = ofc.ztaac.dataShare.getIntrinsicZk(FilterType.REF, ofc.ztaac.dataShare.getFieldIdx(detectorList))

In [ ]:
add_coma_x = np.array(intrinsic_zk, copy=True)
sub_coma_x = np.array(intrinsic_zk, copy=True)

In [ ]:
add_coma_x[:,3:4] += 0.2  # add 0.1 um of vertical coma
# sub_coma_x = np.zeros_like(intrinsic_zk)
sub_coma_x[:,3:4] -= 0.1  # subtract 0.1 um of vertical coma

In [ ]:
listOfWfErr_add_comax = []
listOfWfErr_sub_comax = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma_x):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_add_comax.append(sensorWavefrontError)


for sensorId, annularZernikePoly in zip(sensorIdList, sub_coma_x):
    print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_sub_comax.append(sensorWavefrontError)


In [ ]:
(
    m2HexapodCorrection_addcx,
    cameraHexapodCorrection_addcx,
    m1m3Correction_addcx,
    m2Correction_addcx,
) = ofc_1.calculateCorrections(listOfWfErr_add_comax)


In [ ]:
(
    m2HexapodCorrection_addcx_2,
    cameraHexapodCorrection_addcx_2,
    m1m3Correction_addcx_2,
    m2Correction_addcx_2,
) = ofc_1.calculateCorrections(listOfWfErr_add_comax)


In [ ]:
(
    m2HexapodCorrection_subcx,
    cameraHexapodCorrection_subcx,
    m1m3Correction_subcx,
    m2Correction_subcx,
) = ofc_1.calculateCorrections(listOfWfErr_sub_comax)

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(m2HexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(cameraHexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_subcx.getCorrection())+\
np.array(cameraHexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())+\
np.array(m2HexapodCorrection_subcx.getCorrection())+\
np.array(cameraHexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_subcx.getCorrection())+\
np.array(cameraHexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())+\
np.array(m2HexapodCorrection_subcx.getCorrection())+\
np.array(cameraHexapodCorrection_subcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx_2.getCorrection())+\
np.array(cameraHexapodCorrection_addcx_2.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())+\
np.array(m2HexapodCorrection_addcx_2.getCorrection())+\
np.array(cameraHexapodCorrection_addcx_2.getCorrection())

In [ ]:
np.array(m2HexapodCorrection_addcx.getCorrection())+\
np.array(cameraHexapodCorrection_addcx.getCorrection())